In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
! pip install pytorch-lightning --quiet
! pip install lightning-bolts --quiet

In [4]:
import torch
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelSummary
from torch import nn
from torch.nn import functional as F
from pl_bolts.models.autoencoders.components import(
resnet18_decoder, resnet18_encoder)

/opt/conda/lib/python3.10/site-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/opt/conda/lib/python3.10/site-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/opt/conda/lib/python3.10/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/opt/conda/lib/python3.10/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:92: UnderReviewWarning: The featu

In [5]:
Batch_size = 64

train_dataset = CIFAR10(root='./data', train= True, transform= transforms.ToTensor(), download = True)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=Batch_size, shuffle=True)

test_dataset = CIFAR10(root='./data', train= False, transform= transforms.ToTensor(), download = True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=Batch_size, shuffle=True)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse','ship','truck')

Files already downloaded and verified
Files already downloaded and verified


In [6]:
next(iter(train_dataloader))[0].shape

torch.Size([64, 3, 32, 32])

In [7]:
next(iter(train_dataloader))[1].shape

torch.Size([64])

In [8]:
y_val = next(iter(train_dataloader))[1]
y = F.one_hot(y_val, num_classes = 10)

In [9]:
y[0]

tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

In [10]:
class VAE(pl.LightningModule):
    """
    Lightning Module for VAE
    """
    def __init__(self, enc_out_dim=256, latent_dim=512, input_height=32):
        """
        Constructor
        """
        # Initialize the lightning module
        super().__init__()

        # Save the input params
        self.save_hyperparameters()

        # Encoder Layers
        self.encoder_conv_layers = nn.Sequential(
            self.conv_layer(in_channels=4, out_channels=64),   # Input Channels = 4  [3 for RGB and 1 for label data]
            self.conv_layer(in_channels=64, out_channels=128),
            self.conv_layer(in_channels=128, out_channels=256)
        )

        # distribution parameters
        self.fc_mu = nn.Linear(enc_out_dim * 26 * 26, latent_dim)    # 26 since input image size is reduced to 26 after 3 conv layers
        self.fc_var = nn.Linear(enc_out_dim * 26 * 26, latent_dim)

        # Decoder's Linear Layer
        self.linear = nn.Linear(latent_dim + 10, enc_out_dim * 26 * 26)  # 10 [One for each class]

        # Decoder Layers
        self.decoder_conv_layers = nn.Sequential(
            self.transpose_layer(in_channels=256, out_channels=128),
            self.transpose_layer(in_channels=128, out_channels=64),
            self.transpose_layer(in_channels=64, out_channels=3),
        )

        # for the gaussian likelihood
        self.log_scale = nn.Parameter(torch.Tensor([0.0]))

    def conv_layer(self, in_channels, out_channels):
        """
        Function to return conv layer
        """
        return nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, bias=False),
            nn.ReLU()
        )

    def transpose_layer(self, in_channels, out_channels):
        """
        Function to return Transpose layer
        """
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=3, bias=False),
            nn.ReLU()
        )

    def encoder(self, x, y):
        """
        Encoder Block of the VAE
        """
        # Batch size and input dimensions
        batch_size = x.shape[0]
        img_height = x.shape[2]   # Height = Width

        # Concatenate Label data to the image data
        y = torch.argmax(y, dim=1).reshape((y.shape[0], 1, 1, 1))
        y = torch.ones((batch_size, 1, img_height, img_height)).to(self.device) * y
        concat_input = torch.cat((x, y), dim=1)

        # Pass the concatenated input through the encoder layers and flatten it
        x = self.encoder_conv_layers(concat_input)
        x = x.view(batch_size, -1)
        return x

    def decoder(self, z, y):
        """
        Decoder Block of VAE
        """
        # Add 10 neurons (one for each class to the latent layer)
        z = torch.cat((z, y.float()), dim=1)

        # Latent layer
        x_hat = F.relu(self.linear(z))
        x_hat = x_hat.reshape(-1, 256, 26, 26)

        # Decoder layers
        x_hat = self.decoder_conv_layers(x_hat)
        x_hat = torch.sigmoid(x_hat)
        return x_hat

    def configure_optimizers(self):
        """
        Optimizer for model training
        """
        return torch.optim.Adam(self.parameters(), lr=1e-4)

    def gaussian_likelihood(self, mean, logscale, sample):
        """
        """
        scale = torch.exp(logscale)
        dist = torch.distributions.Normal(mean, scale)
        log_pxz = dist.log_prob(sample)
        return log_pxz.sum(dim=(1, 2, 3))

    def kl_divergence(self, z, mu, std):
        """
        """
        # --------------------------
        # Monte carlo KL divergence
        # --------------------------
        # 1. define the first two probabilities (in this case Normal for both)
        p = torch.distributions.Normal(torch.zeros_like(mu), torch.ones_like(std))
        q = torch.distributions.Normal(mu, std)

        # 2. get the probabilities from the equation
        log_qzx = q.log_prob(z)
        log_pz = p.log_prob(z)

        # kl
        kl = (log_qzx - log_pz)
        kl = kl.sum(-1)
        return kl

    def training_step(self, batch, batch_idx):
        """
        Function to train the model
        """
        # Input data
        x, y = batch

        # One-Hot encoding of label data
        y = F.one_hot(y, num_classes=10)

        # encode x to get the mu and variance parameters
        x_encoded = self.encoder(x, y)

        # Encoder output mu and sigma
        mu, log_var = self.fc_mu(x_encoded), self.fc_var(x_encoded)

        # sample z from q
        std = torch.exp(log_var / 2)
        q = torch.distributions.Normal(mu, std)
        z = q.rsample()

        # decoded
        x_hat = self.decoder(z, y)

        # reconstruction loss
        recon_loss = self.gaussian_likelihood(x_hat, self.log_scale, x)

        # kl
        kl = self.kl_divergence(z, mu, std)

        # elbo
        elbo = (kl - recon_loss)
        elbo = elbo.mean()

        self.log_dict({
            'elbo': elbo,
            'kl': kl.mean(),
            'recon_loss': recon_loss.mean(),
            'reconstruction': recon_loss.mean(),
            'kl': kl.mean(),
        })

        return elbo

In [ ]:
# Seed for deterministic results
pl.seed_everything(8)

# Instance of the model
model = VAE()

# Trainer configuration
trainer = pl.Trainer(
    callbacks=[ModelSummary(max_depth=1)],
    gpus=1,
    num_sanity_val_steps=1,
    max_epochs=10
    )

# Train the model
trainer.fit(model, train_dataloader, test_dataloader)

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:106: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")


Training: 0it [00:00, ?it/s]

In [ ]:
torch.save(model.state_dict(), 'vae_cifar_model.pth')

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow, figure
import numpy as np


def vae_results(x, y, model, classes, correct_labels=True, num_outputs=25):
    """
    Function to display output images generated by the model
    """
    # Figure to display the results
    figure(figsize=(8, 3), dpi=300)
    device = model.device

    # Set the model to eval mode
    with torch.no_grad():
        # Change the value to get incorrect label values
        y_incorrect = y - 1                   # Reduce class value by 1
        y_incorrect[y_incorrect == -1] = 9    # Change -1 to 9

        # One-Hot Encoding
        one_hot_y = F.one_hot(y, num_classes=10)
        one_hot_incorrect_y = F.one_hot(y_incorrect, num_classes=10)

        # Send correct labels to Encoder based on function argument
        if correct_labels:
            x_encoded = model.encoder(x.to(device), one_hot_y.to(device))
        else:
            x_encoded = model.encoder(x.to(device), one_hot_incorrect_y.to(device))

        # Get mean and variance from encoder output
        mu, log_var = model.fc_mu(x_encoded), model.fc_var(x_encoded)

        # Calculate standard deviation
        std = torch.exp(log_var/2)
        q = torch.distributions.Normal(mu,std)
        z = q.rsample()

        # Send correct labels to Decoder based on function argument
        if correct_labels:
            x_hat = model.decoder(z, one_hot_y.to(device))
        else:
            x_hat = model.decoder(z, one_hot_incorrect_y.to(device))

        # Plot Results
        fig = plt.figure(figsize=(10,10))
        for index in np.arange(num_outputs):
            axs = fig.add_subplot(5, 5, index + 1, xticks=[], yticks=[])
            img = x_hat[index].to('cpu')
            plt.imshow(img.permute(1, 2, 0))

            if correct_labels:
                axs.set_title(f"Label: {classes[y[index]]}")
            else:
                axs.set_title(f"Incorrect Label: {classes[y_incorrect[index]]}\n Correct Label: {classes[y[index]]}")

        fig.tight_layout()
        plt.show()

In [ ]:
x, y = next(iter(test_dataloader))
vae_results(x, y, model, classes)

In [ ]:
vae_results(x, y, model, classes, correct_labels=False)

In [ ]:
vae_results(x, y, model, classes, correct_labels=True)